In [75]:
import json
import cv2
import os
import matplotlib.pyplot as plt
import glob
import copy
import random
import numpy as np
from scipy import ndarray, ndimage
import splitfolders

In [2]:
print(os.listdir())

['.DS_Store', '.ipynb_checkpoints', 'augment.ipynb', 'images', 'images_augmented', 'JsonTask2and3 (1).ipynb', 'parser_file (1).ipynb', 'parser_file (2).ipynb', 'parser_file.ipynb', 'Untitled15.ipynb', 'yolo_annotation']


In [48]:
import os
import cv2
import glob
import json
import copy
import random
import numpy as np
from scipy import ndarray, ndimage

def json_from_dir(source_dir):
    all_jsons = glob.glob(os.path.join(source_dir,"*.json"))
    if all_jsons:
        all_jsons = {p:os.path.getmtime(p) for p in all_jsons}
        return sorted(all_jsons.items())[0][0]
    return None


def rotate(image_array: ndarray, angle):
    return ndimage.rotate(image_array, angle, reshape=True, order=0)


def augment_image(img, angle, im_name):
    augmented = rotate(img, angle)
    name_parts = im_name.split(".")
    name, ext = name_parts[0], name_parts[-1]
    aug_im_name = name + "_" + str(angle) + "." + ext
    return augmented, aug_im_name


def get_and_adjust_rotation_matrix(angle, cx, cy, h, w):
    M = cv2.getRotationMatrix2D((cx, cy), angle, 1.0)
    cosang = np.abs(M[0, 0])
    sinang = np.abs(M[0, 1])
    nW = int((h * sinang) + (w * cosang))
    nH = int((h * cosang) + (w * sinang))
    M[0, 2] += (nW / 2) - cx
    M[1, 2] += (nH / 2) - cy
    return M


def rotatePolygon(corners, M, w, h, ln=8):
    corners = corners.reshape(-1,2)
    corners = np.hstack((corners, np.ones((corners.shape[0],1), dtype = type(corners[0][0]))))
    calculated = np.dot(M,corners.T).T
    calculated = calculated.reshape(-1,ln)
    if len(np.where(calculated[:, range(0,ln,2)] > w)) == 0 or len(np.where(calculated[:, range(1,ln,2)] > h)) == 0:
        return None
    return calculated


def transform_regions(regions, rotation_matrix, w, h):
    r_list = []
    for region in regions:
        label = list(region["region_attributes"]["class"])[0]
        x1 = region["shape_attributes"]["x"]
        y1 = region["shape_attributes"]["y"]
        width = region["shape_attributes"]["width"]
        height = region["shape_attributes"]["height"]
        x2 = width + x1
        y2 = height + y1
        nl = np.array([x1,y1,x2,y1,x2,y2,x1,y2])
        p = rotatePolygon(np.array([nl]), rotation_matrix, w, h, ln=len(nl))[0]
        r_list += [
            {
                'shape_attributes': {
                    'name': 'polygon',
                    "all_points_x": [p[i] for i in range(0,len(p),2)],
                    "all_points_y": [p[i] for i in range(1,len(p),2)]
                },
                'region_attributes': {
                    'class': {
                        label: True
                    }
                }
            }
        ]
    return r_list


In [78]:
data=open('images_augmented/annoatations.json').read()
data=json.loads(data)
# print(data)
def get_final_box(corners):
    #main_array = np.array(calculated) # converting to numpy array
    x_ = corners[:4]
    y_ = corners[4:]
    
    xmin = np.min(x_).reshape(-1,1)
    ymin = np.min(y_).reshape(-1,1)
    xmax = np.max(x_).reshape(-1,1)
    ymax = np.max(y_).reshape(-1,1)
    
    ymax=ymax-ymin
    xmax=xmax-xmin
    final = np.hstack((xmin, ymin, xmax, ymax))
    
    return final
for x in list(data['_via_img_metadata'].values()):
    filename = x['filename']
    for region in x['regions']:
        all_points_x = region['shape_attributes']['all_points_x']
        all_points_y = region['shape_attributes']['all_points_y']
        class_name = list(region['region_attributes']['class'].keys())[0]
        #print(filename, all_points_x, all_points_y,class_name)
        all_points_x.extend(all_points_y)

In [79]:
def get_images():
    images=os.listdir('images_augmented')
#     print(images)
    temp={}
    for i in images:
        try:
            temp[i] = cv2.imread(f'images_augmented/{i}')
        except Exception as E:
            pass
    return temp

In [80]:
images_data = get_images()

In [76]:
root_dir = "images"
dest_dir = root_dir.rstrip("/") + "_augmented"

if not os.path.exists(dest_dir):
    os.mkdir(dest_dir)

json_path = json_from_dir(root_dir)
with open(json_path) as f:
    annotations_data = json.load(f)

new_annotations_data = {
    "_via_settings": copy.deepcopy(annotations_data["_via_settings"]),
    "_via_img_metadata": {},
    "_via_attributes": copy.deepcopy(annotations_data["_via_attributes"])
}

count = 0
for im_key in annotations_data["_via_img_metadata"]:
    count += 1
    print("processing image "+str(count)+"/"+str(len(annotations_data["_via_img_metadata"]))+" ...")
    im_name = annotations_data["_via_img_metadata"][im_key]["filename"]
    im_path = os.path.join(root_dir,im_name)
    if os.path.exists(im_path):
        img = cv2.imread(im_path)
        if len(img.shape) is 2:
            img = gray2rgb(img)
        img = img[:,:,:3]
        height, width = img.shape[0],img.shape[1]
        cx = width//2
        cy = height//2
        for angle in range(0, 360, 2):
            augmented, aug_im_name = augment_image(img, angle, im_name)
            aug_im_path = os.path.join(dest_dir, aug_im_name)
            cv2.imwrite(aug_im_path, augmented)
            rotation_matrix = get_and_adjust_rotation_matrix(angle, cx, cy, height, width)
            aug_size = os.path.getsize(aug_im_path)
            aug_im_key = aug_im_name + str(aug_size)
            new_annotations_data["_via_img_metadata"][aug_im_key] = {
                "filename": aug_im_name,
                "size": aug_size,
                "regions": transform_regions(annotations_data["_via_img_metadata"][im_key]["regions"], rotation_matrix, width, height),
                "file_attributes": {}
            }

aug_json_path = os.path.join(dest_dir,"annoatations.json")
with open(aug_json_path,"w") as f:
    json.dump(new_annotations_data,f)
    print("Done saving in "+dest_dir)
    


<>:25: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:25: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-76-a1e9826caa3c>:25: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if len(img.shape) is 2:


processing image 1/7 ...
processing image 2/7 ...
processing image 3/7 ...


Copying files: 0 files [1:30:04, ? files/s]


processing image 4/7 ...
processing image 5/7 ...
processing image 6/7 ...
processing image 7/7 ...
Done saving in images_augmented


In [95]:
os.chdir('C:/Users/Abdul Basit/Downloads/augmentations_task/images_augmented')
if os.path.exists('annoatations.json'):
    def normalize_for_yolo(xmin, ymin, w, h, w_img, h_img):
        xcenter = (xmin + w/2) / w_img
        ycenter = (ymin + h/2) / h_img
        w = w / w_img
        h = h / h_img
        return xcenter,ycenter,w,h


    count = 0
    class_dt = {}
    
    os.chdir('C:/Users/Abdul Basit/Downloads/augmentations_task')
    if not os.path.exists('yolo_annotation'):
        os.mkdir('yolo_annotation')

    for x in list(data['_via_img_metadata'].values()):
        filename = x['filename']
        u_filename = filename.split('.')[0]
        width, height = (images_data[filename].shape)[:2]
        f = open(f'yolo_annotation/{u_filename}.txt', "w")

        for region in x['regions']:
            all_points_x = region['shape_attributes']['all_points_x']
            all_points_y = region['shape_attributes']['all_points_y']
            class_name = list(region['region_attributes']['class'].keys())[0]
            if class_dt.__contains__(class_name)==False:
                class_dt[class_name] = count
                count += 1

    #         print(filename, all_points_x, all_points_y,class_name)
            all_points_x.extend(all_points_y)
            four_points = get_final_box(all_points_x)
            x, y, w, h = four_points[0] 
            x, y, w, h = normalize_for_yolo(x, y, w, h, width, height)

            f.write(f"{class_dt[class_name]} {x} {y} {w} {h}\n")
    #         break
        f.close()

    


In [ ]:
def  making_directories():
    if not os.path.exists('split'):
        os.mkdir('split')
    os.chdir('split')
    if not os.path.exists('images_augmented'):
        os.mkdir('images_augmented')
    if not os.path.exists('yolo_annotation'):
        os.mkdir('yolo_annotation')
def copying_images():
    os.chdir("C:\\Users\\Abdul Basit\\Downloads\\augmentations_task")
    for files in glob('images_augmented/*.jpeg'):
        shutil.copyfile(f'{files}', f'split/{files}')
def copying_text():
    for files in glob('yolo_annotation/*.txt'):
        shutil.copyfile(f'{files}', f'split/{files}')
def splitting_train_test():
    splitfolders.ratio('split', output="output", seed=1337, ratio=(.8, 0.2))
    
    

In [138]:
os.chdir("C:\\Users\\Abdul Basit\\Downloads\\augmentations_task")
if len(os.listdir('images_augmented'))-1==len(os.listdir("yolo_annotation")):
    making_directories()
    copying_images()
    copying_text()
    splitting_train_test()
    !git clone https://github.com/ultralytics/yolov5.git
    %cd yolov5/
    !pip install -r requirements.txt
    !python train.py --img 600 --batch 20 --epochs 5 --data coco128.yaml --weights yolov5s.pt --nosave --cache
    

In [14]:
# !pip install split-folders

In [108]:
# shutil.make_archive('output_f', 'zip', 'output')

'C:\\Users\\Abdul Basit\\Downloads\\augmentations_task\\output_f.zip'

In [141]:
# if not os.path.exists('dataset'):
#     os.mkdir('dataset')
#     print('a')

In [119]:
# shutil.unpack_archive('output_f.zip', 'C:\\Users\\Abdul Basit\\Downloads\\augmentations_task\\dataset')